In [122]:
import numpy as np
from machinedesign.model_builders import fully_connected
from machinedesign.viz import grid_of_images_default
from machinedesign.autoencoder.interface import iterative_refinement
from skimage.io import imsave
import joblib
from keras.layers import Input, Dense, Flatten, Activation, Lambda, Reshape
from keras.models import Model

In [121]:
data = joblib.load('../results/fcsa_out/fcsa.pkl')
for d in data:
    print(d.shape)
data[0].shape

(784, 400)
(400,)
(400, 900)
(900,)
(900, 100)
(100,)
(100, 200)
(200,)
(200, 800)
(800,)
(800, 784)
(784,)


(784, 400)

In [99]:

"""
params = {
    'nb_hidden_units': [400, 900, 100, 200, 800],
    'activations': ['relu', 'relu', 'relu', 'relu', 'relu'],
    'output_activation': 'sigmoid'
}
model = fully_connected(params, (1, 28, 28), (1, 28, 28))
"""
inp = Input((1, 28, 28))
x = Flatten()(inp)
x = Dense(400)(x)
x = Activation('relu')(x)
x = Dense(900)(x)
x = Activation('relu')(x)
x = Dense(100)(x)
x = Activation('relu')(x)
x = Dense(200)(x)
x = Activation('relu')(x)
x = Dense(800)(x)
#x = Lambda(wta_fc_sparse())(x)
x = Activation('relu')(x)
x = Dense(784)(x)
x = Activation('sigmoid')(x)
x = Reshape((1, 28, 28))(x)
out = x
model = Model(inp, out)

In [89]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1, 28, 28)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 400)               314000    
_________________________________________________________________
activation_25 (Activation)   (None, 400)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 900)               360900    
_________________________________________________________________
activation_26 (Activation)   (None, 900)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 100)               90100     
__________

In [100]:
ptr = 0
for l in [2, 4, 6, 8, 10, 12]:
    layer = model.layers[l]
    W, b = data[ptr], data[ptr + 1]
    Wk, bk = layer.weights
    print(Wk.get_value().shape, W.shape)
    print(bk.get_value().shape, b.shape)
    Wk.set_value(W)
    bk.set_value(b)
    ptr += 2

(784, 400) (784, 400)
(400,) (400,)
(400, 900) (400, 900)
(900,) (900,)
(900, 100) (900, 100)
(100,) (100,)
(100, 200) (100, 200)
(200,) (200,)
(200, 800) (200, 800)
(800,) (800,)
(800, 784) (800, 784)
(784,) (784,)


In [118]:
model.transformers = []
params = {
    'batch_size': 128,
    'nb_samples': 25*25,
    'nb_iter': 100,
    'binarize':{
        'name': 'binary_threshold',
        'params': {
            'one_ratio': 0.13,
            'is_moving': True,
        }
    },
    'noise':{
        'name': 'none',
        'params': {
        }
    },
    'stop_if_unchanged': False,
    'seed': 42,
}
X = iterative_refinement(params, model)
x = X[:, -1]
x =  1 - x
x = grid_of_images_default(x)
imsave('out.png', x)

/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [119]:
model.save('../results/fcsa_out/model.h5')